In [5]:
import os
import operator

import numpy as np
import pandas as pd
import gensim.models
import multiprocessing


from gensim import utils
from unidecode import unidecode
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDiA
from gensim.parsing.preprocessing import strip_punctuation

from gensim.parsing.preprocessing import strip_multiple_whitespaces



import pre


In [ ]:

# txt_test_sample.txt

with open('Original_Forums.txt', mode='r', encoding='iso-8859-1') as f:
    
    # find CarType, MsgTitle, MsgAuther, MsgDate and Msg in file
    car_type = []    # car tyepe: Makt + model
    makes = []
    msg_date = []
    msg_title = []
    msg_author = []
    reviews = [] 
        
    while True:
            
        end_of_message = False  # message end flag init
        
        line = f.readline()
        if not line:
            break
            
        if len(line) == 1:
            # message session end when read blank line
            end_of_message = True
            
        if end_of_message:
            # when the message sessin end, append to review list
            reviews.append(msg)
            
        if line.startswith('CarType:'):
            # get car type line
            type_ = line.split(':')[-1]
            car_type.append(type_)
            try:
                makes.append(type_.split()[1])
            except IndexError:
                # error happened on '71-Mazda6'
                makes.append(type_.split('-')[1].replace('6', ''))
            msg = ''    # init msg      
        elif line.startswith('MsgTitle:'):
            msg_title.append(line.split(':')[-1])
        elif line.startswith('MsgAuther:'):
            msg_author.append(line.split(':')[-1])
        elif line.startswith('MsgDate:'):
            time = line.split('Date:')[-1]
            msg_date.append(time)  
        elif line.startswith(' ') and not end_of_message:
            msg += line
            
    df = pd.DataFrame()
    df['make'] = makes
    df['type'] = car_type
    df['title'] = msg_title
    df['author'] = msg_author
    df['date'] = msg_date
    df['review'] = reviews


for i in range(len(df.columns)):
    # rm '\n' in cells
    df[df.columns[i]] = df[df.columns[i]].apply(lambda x: str(x).replace('\n', ''))
    
# df.to_csv('original_review.csv', index=False)    

In [35]:
df.head()
df['make'].unique()

array(['Acura', 'Audi', 'BMW', 'Buick', 'Cadillac', 'Chevrolet',
       'Chrysler', 'Dodge', 'Dodge/Plymouth', 'Ford', 'Honda', 'Hyundai',
       'Infiniti', 'Jaguar', 'Kia', 'Lexus', 'Lincoln', 'Maserati',
       'Mazda', 'Mercedes-Benz', 'Mercedes', 'Mitsubishi', 'Nissan',
       'Oldsmobile', 'Pontiac', 'Saab', 'Saturn', 'Suzuki', 'Toyota',
       'Volkswagen', 'Volvo'], dtype=object)

In [8]:
# 6 brands for compare, and each brands saved as individual .csv
makes = 'Honda Toyota Nissan Lexus BMW Audi Acura'.split()
makes

['Honda', 'Toyota', 'Nissan', 'Lexus', 'BMW', 'Audi', 'Acura']

We have 7 brands now, 4 luxury and 4 general.

Luxury | General
:-: | :-:
Acura | Honda
BMW | Nissan
Lexus | Toyota
Audi| 

They also have some hidden relations, like Acura is a child company of Honda, and Lexus belong to Toyota, these related brands may have some unusual score than those just in same level.

In [ ]:
for i in makes:
    temp = df[df['make'] == i]
    temp = temp.dropna(axis=0)
    temp.reset_index(inplace=True, drop=True)
    temp.to_csv('data/{}.csv'.format(i), index=None)

        
merged = pd.read_csv('data/csv_data/Honda.csv')
merged = merged.append(pd.read_csv('data/csv_data/Acura.csv'))
merged = merged.append(pd.read_csv('data/csv_data/Toyota.csv'))
merged = merged.append(pd.read_csv('data/csv_data/Nissan.csv'))
merged = merged.append(pd.read_csv('data/csv_data/Lexus.csv'))
merged = merged.append(pd.read_csv('data/csv_data/BMW.csv'))
merged = merged.append(pd.read_csv('data/csv_data/Audi.csv'))
merged.reset_index(inplace=True, drop=True)
merged.to_csv('data/merged7.csv', index=None)
# merged

In [7]:
temp = pd.read_csv('data/csv_data/merged7.csv')
temp

,make,type,title,author,date,review
0,Honda,45-Honda Honda Accord,I Don't Understand [dtownfb],hondacrv1,"Jan 23, 2007 (11:50 am)",Thanks! Also i forgot to say that since i wa...
1,Honda,45-Honda Honda Accord,Does Hawaii Honda Dealers honor internet EW?,hicrvguy,"Jan 24, 2007 (2:34 pm)","Thank God for the internet! First of all, al..."
2,Honda,45-Honda Honda Accord,Does Hawaii Honda Dealers honor internet EW? ...,dwynne,"Jan 24, 2007 (3:11 pm)",I am pretty sure than once you are past the f...
3,Honda,45-Honda Honda Accord,Does Hawaii Honda Dealers honor internet EW? ...,hicrvguy,"Jan 24, 2007 (4:00 pm)",Really? Geez I just got a quote for the Hond...
4,Honda,45-Honda Honda Accord,Does Hawaii Honda Dealers honor internet EW? ...,skap2,"Jan 24, 2007 (6:41 pm)",I am still deciding on Honda HondaCare for ne...
...,...,...,...,...,...,...
401531,Audi,8-Audi Audi S4,M3 v S4,diver110,"Mar 13, 2004 (5:34 am)","FWIW, I have tested 2004s of both and (everyt..."
401532,Audi,8-Audi Audi S4,Diver110 -,kevin111,"Mar 13, 2004 (8:52 pm)","Does the S4 need special Oil, when giving it ..."
401533,Audi,8-Audi Audi S4,"1997 M3, Check Engine Light",97bimmerm3,"Mar 24, 2004 (6:29 am)",I recently purchased a 1997 M3 with 72K miles...
401534,Audi,8-Audi Audi S4,Want to know positives/negatives of S4 and M3,jlmagicarethus,"Feb 05, 2004 (10:24 pm)",I'm looking into getting either a S4 or M3 of...


now, we gonna figure out the total appearence of each brand.

In [12]:
# merged = pd.read_csv('data/merged6.csv')
 
num_review = {}  # count the num of brands reviews
for i in makes:
    temp = pd.read_csv('data/csv_data/{}.csv'.format(i))
    num_review['{}'.format(i)] = temp.shape[0]
print(num_review)

{'Honda': 92130, 'Toyota': 55362, 'Nissan': 31877, 'Lexus': 55192, 'BMW': 88721, 'Audi': 31438, 'Acura': 46816}


$$LIFT = \frac{P(A*B)}{P(A)*P(B)}$$
P(A*B) means the count of A and B co-mentioned.<br>
P(A) and P(B) means count of A or B in reviews 

reference paper: [Mine Your Own Business: Market Structure Surveillance through Text Mining.](http://www.columbia.edu/~on2110/Papers/Netzer_Feldman_Goldenberg_Fresko_2012.pdf).


This function is aiming to calculate the LIFT score when two brands showed together. Once find the target words showed in the,
review simutaneously, counter plus 1, but no matter how many times it appear in one review, it will always counted as 1 time.
Like, Toyota is similar to Honda, but Toyota is more porpular in Dallas, although Toyota appeared twice in this sentence, it will just
count as 1 time.

But the bigest problem is proplr not always called a car with its total name, like Honda civic, peoplr usually ignore the brand
name, these data from a website highly related to cars, users might pretty fimilar with car information. So, those just mentioned 
certain make name, will not be counted.

In [16]:

def bi_gram(makes):
    for outer_make in makes:

        frequency = {}   # count co-mentioned, empty dict 
        lift = {}        # lift score, empty dict
        temp = pd.read_csv('data/csv_data/{}.csv'.format(outer_make))
        num_review['{}'.format(outer_make)] = temp.shape[0]          # denominator

        for inner_make in [make for make in makes if make != outer_make]:        
            counter = 0   # counter for appearence
            combined_name = '{}_{}'.format(outer_make, inner_make)
            
            for review in temp['review']:
                if inner_make in str(review):
                    counter += 1
            frequency[combined_name] = counter      # co-mentioned, numerator
            lift[combined_name+'_lift'] = \
                    round(frequency[combined_name] / \
                    (num_review[combined_name.split('_')[0]] * \
                    num_review[combined_name.split('_')[1]]) * (10**7), 3)     # scale by 10e7 and keep 3 digits

        sorted_lift = sorted(lift.items(), key=operator.itemgetter(1))         # sort dict by value in increading

        print(sorted_lift)
        print('\n')
        
bi_gram(makes)
    

[('Honda_BMW_lift', 1.274), ('Honda_Lexus_lift', 1.554), ('Honda_Audi_lift', 1.947), ('Honda_Nissan_lift', 4.331), ('Honda_Acura_lift', 4.605), ('Honda_Toyota_lift', 7.611)]


[('Toyota_BMW_lift', 1.507), ('Toyota_Audi_lift', 1.85), ('Toyota_Acura_lift', 2.631), ('Toyota_Lexus_lift', 4.608), ('Toyota_Nissan_lift', 5.032), ('Toyota_Honda_lift', 7.56)]


[('Nissan_BMW_lift', 2.365), ('Nissan_Lexus_lift', 2.796), ('Nissan_Audi_lift', 3.273), ('Nissan_Acura_lift', 3.967), ('Nissan_Honda_lift', 10.011), ('Nissan_Toyota_lift', 12.67)]


[('Lexus_Honda_lift', 2.212), ('Lexus_Nissan_lift', 3.78), ('Lexus_Acura_lift', 8.41), ('Lexus_Toyota_lift', 9.913), ('Lexus_BMW_lift', 14.05), ('Lexus_Audi_lift', 14.201)]


[('BMW_Honda_lift', 1.783), ('BMW_Toyota_lift', 2.244), ('BMW_Nissan_lift', 3.069), ('BMW_Acura_lift', 5.564), ('BMW_Lexus_lift', 7.593), ('BMW_Audi_lift', 13.201)]


[('Audi_Honda_lift', 2.583), ('Audi_Toyota_lift', 3.947), ('Audi_Nissan_lift', 4.031), ('Audi_Acura_lift', 9.091), ('Audi

As the result showing above, we can clearly know that the relations between different brands. Like for Honda, it have a obvious high relation with Toyota, which is cater to the normal concept. And hidden relations also worked, Honda_Acura have a higher score than Honda_Nissan<br>

dicsussion|switching
:-:|:-:
![dicsussion](img/discussion.png "disscussion")|![switching](img/switching.png "switching")
                        <a id="Figure1">Figure 1</a>

<br>

In this function, tri-gram will be counted, and all other steps similar to the above bi-gram function. 
Function start at a brand, and then search in the reviews if two other brands sppeared in the review. However, the sequence of 
these two brand is doesn't matter, this cause a problem that some overlapping happened, Honda_Nissan_Toyota, and Honda_Toyota_Nissan
is excately same result actually.

In [18]:

def tri_gram(makes):
    for outer_make in makes:

        frequency = {}   # count comentioned 
        lift = {}
        temp = pd.read_csv('data/csv_data/{}.csv'.format(outer_make))
        num_review['{}'.format(outer_make)] = temp.shape[0]
        new_sort = []
        
        mid_makes = [make for make in makes if make != outer_make]
        for mid_make in mid_makes:
            for inner_make in [make for make in mid_makes if make != mid_make]:
                combined_name = '{}_{}_{}'.format(outer_make, mid_make, inner_make)
                counter = 0   # counter for appearence
    
                for review in temp['review']:
                    if mid_make in str(review):
                        if inner_make in str(review):
                            counter += 1
                frequency[combined_name] = counter
                lift[combined_name+'_lift'] = round(frequency[combined_name] / \
                        (num_review[combined_name.split('_')[0]] * \
                        num_review[combined_name.split('_')[1]] * \
                        num_review[combined_name.split('_')[2]]) * (10**13), 3)   # scale by 10e13 and keep 3 digits
                
        sorted_lift = sorted(lift.items(), key=operator.itemgetter(1))          # sort dict by value
#       the sorted dict will have some over lapping, like Honda_Nissan_Toyota and Honda_Toyota_Nissan, they have same score
#       so, just need keep even index data.
        for i in range(0, len(sorted_lift), 2):
            new_sort.append(sorted_lift[i])
        print(new_sort) 
        print('\n')
        
tri_gram(makes)


[('Honda_Nissan_BMW_lift', 2.648), ('Honda_Lexus_Audi_lift', 2.878), ('Honda_Lexus_BMW_lift', 3.236), ('Honda_Nissan_Audi_lift', 3.358), ('Honda_Toyota_BMW_lift', 3.624), ('Honda_BMW_Acura_lift', 4.338), ('Honda_Nissan_Lexus_lift', 4.442), ('Honda_BMW_Audi_lift', 4.514), ('Honda_Toyota_Audi_lift', 4.74), ('Honda_Audi_Acura_lift', 5.752), ('Honda_Nissan_Acura_lift', 7.491), ('Honda_Lexus_Acura_lift', 8.696), ('Honda_Toyota_Lexus_lift', 9.165), ('Honda_Toyota_Acura_lift', 9.213), ('Honda_Toyota_Nissan_lift', 34.689)]


[('Toyota_Nissan_BMW_lift', 3.321), ('Toyota_Lexus_Audi_lift', 3.331), ('Toyota_BMW_Acura_lift', 3.349), ('Toyota_Honda_BMW_lift', 3.602), ('Toyota_BMW_Audi_lift', 4.145), ('Toyota_Audi_Acura_lift', 4.295), ('Toyota_Nissan_Audi_lift', 4.506), ('Toyota_Honda_Audi_lift', 4.615), ('Toyota_Lexus_BMW_lift', 4.722), ('Toyota_Nissan_Acura_lift', 5.931), ('Toyota_Honda_Lexus_lift', 6.536), ('Toyota_Nissan_Lexus_lift', 8.111), ('Toyota_Lexus_Acura_lift', 11.115), ('Toyota_Honda_Acu

The most significant bandary in these 7 brands is like the Figures showed above, Lexus car and General cars have a kind seperate social community. Members in top score of each brand is those more closer in the <a href="#Figure1">Figure 1</a>. 
Also, hidden relations we mentioned is clearly displied, Honda and Toyota was co-mentioned with Lexus or Acura many times. 

In [ ]:
def topic_score_gen(makes):
    for make in makes:
        corpus = []
        temp = pd.read_csv('data/csv_data/{}.csv'.format(make))
        for review in temp['review']:
            corpus.append(pre.string_processor(str(review)))
        count_vect = CountVectorizer()
        df_bow = pd.DataFrame(data=count_vect.fit_transform(raw_documents=corpus).toarray())
        column_nums, terms = zip(*sorted(zip(count_vect.vocabulary_.values(), count_vect.vocabulary_.keys())))
        df_bow.column_nums = terms
                          
        ldia = LDiA(n_components=10)
        ldia.fit_transform(df_bow)
                          
        df_topics = pd.DataFrame(data=ldia.components_.T, index=terms)
        df_topics.to_csv('data/topics/{}_topic.csv'.format(make))
                          

def topic_word_gen(makes):
    for make in makes:
        df = pd.read_csv('data/topics/{}_topic.csv'.format(make), index_col='Unnamed: 0')

        df_words = pd.DataFrame()
        for i in df.columns:
            df_words['File{}'.format(i)] = list(df[i].nlargest(9).index)

        df_words.to_csv('data/topics/{}_topic_word.csv'.format(make), index=False)
    
topic_score_gen(makes)
topic_word_gen(makes)


In [20]:
a = pd.read_csv('data/topics/BMW_topic_word.csv')
a

,File0,File1,File2,File3,File4,File5,File6,File7,File8,File9
0,tire,bmw,car,look,nbsp,price,engine,car,drive,bmw
1,wheel,car,time,com,bmw,dealer,the,bmw,car,car
2,sport,dealer,get,black,lease,lease,fuel,audi,speed,lexus
3,winter,service,day,thank,car,deal,gas,new,the,the
4,package,mile,go,color,vehicle,invoice,power,like,manual,like
5,snow,problem,drive,leather,host,car,car,year,awd,good
6,the,warranty,the,http,prices,msrp,torque,think,bmw,drive
7,car,the,week,www,forum,good,hp,good,like,acura
8,mile,oil,like,post,year,month,high,buy,road,infiniti


In [21]:
ps = PorterStemmer()

def simple_processor(token):
    str = unidecode(token)
    str = strip_punctuation(str)
    str = strip_multiple_whitespaces(str)

    tokens = str.split(' ')
    tokens = [ps.stem(token) for token in tokens]
    str = ' '.join(tokens)
    return str

def re_pre(makes):
    for make in makes:
        df = pd.read_csv('data/csv_data/{}.csv'.format(make))

        corpus = []
        for review in df['review']:
            corpus.append(simple_processor(str(review)))

        with open('data/corpus/{}.txt'.format(make), mode='w', encoding='utf-8') as f:
            for line in corpus:
                f.write(line + '\n')
            
re_pre(makes)

In [24]:
worker = multiprocessing.cpu_count()    # num of processors

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""
        
    def __iter__(self):
        for line in open(path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)
            
            
path_list = ['data/corpus/{}.txt'.format(make) for make in makes]
for path_ in path_list:
    path = path_
    sentence = MyCorpus()
    model_w2v = gensim.models.Word2Vec(sentences=sentence,
                                       workers=worker,
                                      sg=0,
                                      size=300)

    model_name = '{}_W2V_model'.format(path.split('/')[-1].split('.')[0])
    model_w2v.save('pretrained/{}'.format(model_name))

In [28]:
ps = PorterStemmer()

model_path = 'pretrained/Honda_W2V_model'
model_w2v = gensim.models.Word2Vec.load(model_path)
model_w2v.most_similar('{}'.format(ps.stem('Accura')), topn=20)

[('rs', 0.558880090713501),
 ('rodeo', 0.5536184310913086),
 ('fx', 0.548690676689148),
 ('kappa', 0.5430014133453369),
 ('nsx', 0.53934645652771),
 ('rl', 0.5325500965118408),
 ('murano', 0.5292455554008484),
 ('gs', 0.5244740843772888),
 ('cl', 0.521479606628418),
 ('citat', 0.5214464664459229),
 ('lexus', 0.5206010341644287),
 ('sh', 0.5205627679824829),
 ('rdx', 0.5198386907577515),
 ('antra', 0.5197892189025879),
 ('infin', 0.5187973976135254),
 ('nuvi', 0.511254072189331),
 ('futher', 0.5107113718986511),
 ('amg', 0.5091148018836975),
 ('avalon', 0.5088338851928711),
 ('soundgat', 0.5080039501190186)]

In Word2Vect model, it goes deeply with the relations, not only just brands, certain models showed mostly. Like the in the Lift score of Acura, the most related brand is Audi, and RS is the high performance car of Audi, Murano is from Nissan, the absence of BMW and Lexus may caused by the mis-count on model name.